# Testing and Profiling

## Timing Python Code

In [1]:
import time
import timeit

In [2]:
start_time = time.time()

fact = 1
for i in range(1, 10):
    print(i)
    fact *= i
    
end_time = time.time()

print(fact)
print("run_time: %f" % (end_time - start_time))

1
2
3
4
5
6
7
8
9
362880
run_time: 0.000368


In [3]:
def my_factorial():
    fact = 1
    for i in range(1, 1001): 
        fact *= i
    return fact

k = 10_000
print(timeit.timeit(my_factorial, number=k) / k) #time it - specify how many times and the average

0.00023442493081092834


In [4]:
%timeit my_factorial() # handy magic commmand % line vs. %% cell

%timeit -n 100 my_factorial() # standard deviation, see memit (memory useage)

238 µs ± 2.87 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
238 µs ± 2.76 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Profiling Python Code

In [ ]:
import cProfile
%load_ext snakeviz

In [25]:
def my_factorial(n):
    fact = 1
    for i in range(1, n): 
        fact *= i
    return fact

In [26]:
%prun my_factorial(1000)

         4 function calls in 0.001 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.001    0.001 <ipython-input-25-6f2bffb074f0>:1(my_factorial)
        1    0.000    0.000    0.001    0.001 {built-in method builtins.exec}
        1    0.000    0.000    0.001    0.001 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

 - ncalls: Total number of calls to the function. If there are two numbers, that means the function recursed and the first is the total number of calls and the second is the number of primitive (non-recursive) calls.
 - tottime: Total time spent in the function, not including time spent in calls to sub-functions
 - percall: tottime divided by ncalls
 - cumtime: Cumulative time spent in this function and all sub-functions
 - percall: cumtime divided by ncalls
 - filename:lineno(function): File name and line number were the function is defined, and the function’s name

In [33]:
%%snakeviz

my_factorial(1000)

my_factorial(10)

 
*** Profile stats marshalled to file 'C:\\Users\\policast\\AppData\\Local\\Temp\\tmpc6vguft8'. 
Embedding SnakeViz in this document...


## Testing 

In [45]:
import numpy as np

import unittest
import doctest
from typing import Tuple
from hypothesis import given, assume, strategies as st

In [37]:
def quadratic(a: float, b: float, c: float) -> Tuple[complex, complex]:
    ''' Compute the roots of the quadratic equation:
            ax^2 + bx + c = 0
        Written in Python as:
            a*x**2 + b*x + c == 0.0

    Input
    =====
    a: float 
    b: float 
    c: float
    
    Output
    ======
    x1: complex
    x2: complex
    
    Example
    =======
    x1, x2 = quadratic(a=1, b=0, c=1)
    Here x1 = 1 and x2 = -1
    
    '''
    discriminant = np.sqrt(b**2.0 - 4.0*a*c)
    x1 = (-b + discriminant) / (2.0 * a)
    x2 = (-b - discriminant) / (2.0 * a)
    return x1, x2


#### unitttest

In [54]:
class TestSum(unittest.TestCase):

    def test_quadratic(self):
        x1, x2 = quadratic(a=8, b=22, c=15)
        self.assertEqual(x1, -1.25, "Should be -1.25")

unittest.main(argv=[''], verbosity=2, exit=False)

test_quadratic (__main__.TestSum) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


#### doctest

In [52]:
def quadratic(a: float, b: float, c: float) -> Tuple[complex, complex]:
    ''' Compute the roots of the quadratic equation:
            ax^2 + bx + c = 0
        Written in Python as:
            a*x**2 + b*x + c == 0.0
        For example:
            >>> x1, x2 = quadratic(a=8, b=22, c=15)
            >>> x1
            -1.25
            >>> x2
            -1.5
            >>> 8*x1**2 + 22*x1 + 15
            0.0
            >>> 8*x2**2 + 22*x2 + 15
            0.0
    '''
    discriminant = np.sqrt(b**2.0 - 4.0*a*c)
    x1 = (-b + discriminant) / (2.0 * a)
    x2 = (-b - discriminant) / (2.0 * a)
    return x1, x2

print(doctest.testmod())

TestResults(failed=0, attempted=5)


#### hypothesis

In [58]:
@given(a = st.floats(min_value=-10000, max_value=10000),
       b = st.floats(min_value=-10000, max_value=10000),
       c = st.floats(min_value=-10000, max_value=10000))
def test_quad(a, b, c):
    assume(abs(a) >= 0.0001)
    x1, x2 = quadratic(a, b, c)
    
    assert np.isclose(a*x1**2 + b*x1 + c, 0.0, atol=0.0001)
    assert np.isclose(a*x1**2 + b*x1 + c, 0.0, atol=0.0001)
    
test_quad()    

C:\Users\policast\Programs\Anaconda3.7\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in sqrt


Falsifying example: test_quad(
    a=0.00010000000161269897, b=0.0, c=2.2204460492503135e-12,
)


AssertionError: 